# ML4Science Examples

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/holl-/ML4Science/blob/main/docs/Examples.ipynb)
&nbsp; • &nbsp; [🌐 **ML4Science**](https://github.com/holl-/ML4Science)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/ML4Science/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/ML4Science/ml4s)
&nbsp; • &nbsp; [**▶ Videos**]()

All examples listed in this notebook can be run with any of the three supported ML backends: `jax`, `torch` and `tensorflow`.
You can select your preferred one by changing the below `math.use()` call.

In [ ]:
# !pip install ml4s
from ml4s import math, nn
math.use('torch')

### Training an MLP

The following script trains an [MLP](https://holl-.github.io/ML4Science/ml4s/nn#ml4s.nn.mlp) with three hidden layers to learn a noisy 1D sine function in the range [-2, 2].

In [ ]:
net = nn.mlp(1, 1, layers=[128, 128, 128], activation='ReLU')
optimizer = nn.adam(net, learning_rate=1e-3)

data_x = math.random_uniform(math.batch(batch=128), low=-2, high=2)
data_y = math.sin(data_x) + math.random_normal(math.batch(batch=128)) * .2

def loss_function(x, y):
    return math.l2_loss(y - math.native_call(net, x))

for i in range(100):
    loss = nn.update_weights(net, optimizer, loss_function, data_x, data_y)
    print(loss)

We didn't even have to import `torch` in this example since all calls were routed through ML4Science.

### Pairwise Distances

The following function takes a (possibly batched) tensor of positions and computes the distance matrix.

In [ ]:
from ml4s import math  # uses NumPy by default

def pairwise_distances(x: math.Tensor):
    dx = math.rename_dims(x, 'points', 'others') - x
    return math.vec_length(dx)

x = math.random_normal(math.instance(points=3), math.channel(vector="x,y"))
math.print(pairwise_distances(x))

Inside `pairwise_distances`, we rename 'points' to 'others'.
When taking the difference, ML4Science automatically expands both operands by the missing dimensions, adding 'points' to the first argument and 'others' to the second.
An explanation of this automatic reshaping is given [here](https://holl-.github.io/ML4Science/Shapes.html).

### Automatic Differentiation

Next, let's compute the [gradient](https://holl-.github.io/ML4Science/Autodiff.html) of some function of *(x,y)* w.r.t. *x*.

In [ ]:
from ml4s import math
math.use('jax')

def function(x, y):
    return x ** 2 * y

gradient_x = math.gradient(function, wrt='x', get_output=False)
print(gradient_x(2, 1))

### JIT compilation

ML4Science provides two types of [JIT compilation](https://holl-.github.io/ML4Science/JIT.html): the generic [`jit_compile`](https://holl-.github.io/ML4Science/ml4s/math#ml4s.math.jit_compile) calls the corresponding library function while [`jit_compile_linear`](https://holl-.github.io/ML4Science/ml4s/math#ml4s.math.jit_compile_linear) builds an [explicit representation for linear functions](https://holl-.github.io/ML4Science/Matrices.html#Tracing).

In [ ]:
from ml4s import math
math.use('tensorflow')

@math.jit_compile(auxiliary_args='divide_by_y')
def function(x, y, divide_by_y=False):
    if divide_by_y:
        return x ** 2 / y
    else:
        return x ** 2 * y

function(2, 2, False)

Here, we declare `divide_by_y` as an auxiliary argument to force the function to be re-traced when its value changes.
Otherwise, its concrete value would not be available inside the function and could not be used within an `if` clause.

JIT compilation of linear functions is also supported on NumPy.

In [ ]:
from ml4s import math
math.use('numpy')

@math.jit_compile_linear(auxiliary_args='compute_laplace')
def optional_sp_grad(x, compute_gradient):
    if compute_gradient:
        return math.spatial_gradient(x)
    else:
        return -x

optional_sp_grad(math.linspace(0, 1, math.spatial(x=10)), True)

Here, an [explicit sparse matrix representation](https://holl-.github.io/ML4Science/Matrices.html#Tracing) of `optional_sp_grad` is computed each time a new value of `compute_gradient` is passed.


### Solving a sparse linear system with preconditioners

ML4Science supports [solving dense as well as sparse linear systems](https://holl-.github.io/ML4Science/Linear_Solves.html) and can [build an explicit matrix representation from linear Python functions](https://holl-.github.io/ML4Science/Matrices.html) in order to compute preconditioners.
We recommend using ML4Science's tensors, but you can pass native tensors to [`solve_linear()`](https://holl-.github.io/ML4Science/ml4s/math#ml4s.math.solve_linear) as well.
The following example solves the 1D Poisson problem ∇x = b with b=1 with incomplete LU decomposition.

In [ ]:
from ml4s import math
import numpy as np

def laplace_1d(x):
    return math.pad(x[1:], (0, 1)) + math.pad(x[:-1], (1, 0)) - 2 * x

b = np.ones((6,))
solve = math.Solve('scipy-CG', rel_tol=1e-5, x0=0*b, preconditioner='ilu')
sol = math.solve_linear(math.jit_compile_linear(laplace_1d), b, solve)

Decorating the linear function with [`math.jit_compile_linear`](https://holl-.github.io/ML4Science/ml4s/math#ml4s.math.jit_compile_linear) lets ML4Science compute the sparse matrix inside [`solve_linear()`](https://holl-.github.io/ML4Science/ml4s/math#ml4s.math.solve_linear). In this example, the matrix is a tridiagonal band matrix.
Note that if you JIT-compile the [`math.solve_linear()`](https://holl-.github.io/ML4Science/ml4s/math#ml4s.math.solve_linear) call, the sparsity pattern and incomplete LU preconditioner are [computed at JIT time](https://holl-.github.io/ML4Science/NumPy_Constants.html).
The L and U matrices then enter the computational graph as constants and are not recomputed every time the function is called.

## Further Reading

Check out the [🚀 quickstart guide](Introduction.html) for an introduction into [tensors](Tensors.html) and [dimensions](Shapes.html).

[🌐 **ML4Science**](https://github.com/holl-/ML4Science)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/ML4Science/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/ML4Science/ml4s)
&nbsp; • &nbsp; [**▶ Videos**]()